In [1]:
import pandas as pd
import numpy as np
import ast
from pandas import json_normalize

from tqdm.notebook import tqdm
tqdm.pandas()
import warnings
warnings.filterwarnings('ignore')

In [2]:
# модели
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
from lightgbm import LGBMModel
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.dummy import DummyRegressor

# инструменты
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder


from sklearn.metrics import f1_score

from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
from sklearn.utils import shuffle
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline

pd.set_option('display.max_colwidth', 30)

## Обучение модели

### Загрузка даных

In [25]:
train = pd.read_csv('~/kazan-exp/train_lemm.csv')

In [21]:
test = pd.read_csv('~/kazan-exp/test_lemm.csv')

In [26]:
train.head()

,title,level_1,level_2,level_3,level_4,level_5,lemm,shop_id
0,Зарядный кабель Borofone B...,Электроника,Смартфоны и телефоны,Аксессуары и запчасти,Зарядные устройства и кабели,Кабели,зарядный кабель borofone l...,9031
1,Трусы Sela,Одежда,Женская одежда,Белье и купальники,Трусы,Трусы,трусы sela,18305
2,"Гуашь ""ЮНЫЙ ВОЛШЕБНИК"", 12...",Хобби и творчество,Рисование,"Краски, пигменты","Краски, пигменты","Краски, пигменты",гуашь юный волшебник цвето...,16357
3,Колба для кальяна Крафт (р...,Хобби и творчество,Товары для курения,Кальяны и аксессуары,Колбы,Колбы,колба кальян крафт разный ...,34666
4,"Пижама женская, однотонная...",Одежда,Женская одежда,Домашняя одежда,Пижамы,Пижамы,пижама женский однотонный ...,26389


In [23]:
test.head()

,title,lemm
0,Светодиодная лента Smart l...,светодиодный лента smart s...
1,Стекло ПЛЕНКА керамик мато...,стекло пленка керамика мат...
2,Проводные наушники с микро...,проводной наушник микрофон...
3,"Декоративная табличка ""Пра...",декоративный табличка прав...
4,Подставка под ложку керами...,подставка ложка керамическ...


Заранее подготовим словарь для декодирования целевого признака

In [27]:
target = pd.concat([train['level_5'], train['shop_id']], axis=1)

In [28]:
target_keys = target[['level_5', 'shop_id']].drop_duplicates().sort_values('shop_id')

In [29]:
target_id = dict(target_keys.values)

In [30]:
id_target = dict(target_keys[['shop_id', 'level_5']].values)

In [31]:
id_target

{9: 'Подарочные наборы',
 12: 'Безмены и весы',
 13: 'Органайзеры и разделители',
 18: 'Цветы и натюрморты',
 19: 'Подставки и держатели',
 25: 'Материалы для рукоделия',
 36: 'Щетки и губки',
 37: 'Коннекторы',
 42: 'Подставки для столовых приборов',
 43: 'Коврики для мыши',
 46: 'Мыльницы',
 51: 'Воздушные шары',
 54: 'Аксессуары для спецодежды',
 55: 'Карты памяти',
 63: 'Микроскопы',
 64: 'Гольфы и гетры',
 65: 'Носки и подследники',
 66: 'Полки для ванной',
 67: 'Вилки',
 68: 'Карты памяти',
 71: 'Бюстгальтеры',
 78: 'Воздушные шары',
 79: 'Умные часы',
 80: 'Ключницы настенные',
 86: 'Трусы',
 87: 'Рюкзаки, чехлы, сумки для ноутбуков',
 89: 'Кабели',
 93: 'Корректирующее белье',
 95: 'Римские свечи и фонтаны',
 98: 'Создание украшений',
 99: 'Чехлы',
 100: 'Наборы для покера',
 103: 'Коробки',
 104: 'Игровые наушники',
 106: 'Портативная акустика',
 107: 'Портативная акустика',
 110: 'Гирлянды',
 111: 'Геймпады, джойстики и триггеры',
 119: 'Защитные стекла',
 122: 'Худи',
 123: 

### Векторизация

In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [49]:
from nltk.corpus import stopwords 
import nltk
nltk.download('stopwords') 


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Дмитрий\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [50]:
stop_words = set(stopwords.words('russian')) 

In [51]:
corpus_train = train['lemm'].values.astype('U')
corpus_test = test['lemm'].values.astype('U')

In [52]:
# С помощью TF-IDF
# Эта переменая содержит настройку того,
# как векторайзер будет воспринимать слова

token_pattern_ = r'\w{1,}'
count_tf_idf = TfidfVectorizer(stop_words=list(stop_words), min_df=3,  max_features=None, 
            token_pattern = token_pattern_,
            ngram_range=(1, 2), use_idf=1, smooth_idf=1, sublinear_tf=1,
            )

In [53]:
tf_idf_train = count_tf_idf.fit_transform(corpus_train)
tf_idf_test = count_tf_idf.transform(corpus_test)

In [54]:
tf_idf_train.shape

(91120, 27297)

### Определим самые популярные слова в категориях

In [55]:
levels_id = pd.DataFrame()

In [56]:
# переводим категории в числа
level_1 = train.level_1.factorize()[0]

In [57]:
# Вывел кодирование в отдельную таблицу
levels_id['level_1_id'] = level_1
levels_id['level_1'] = train['level_1']

In [58]:
category_id_df = levels_id[['level_1', 'level_1_id']].drop_duplicates().sort_values('level_1_id')

In [59]:
category_to_id = dict(category_id_df.values)
id_to_category = dict(category_id_df[['level_1_id', 'level_1']].values)

In [60]:
id_to_category

{0: 'Электроника',
 1: 'Одежда',
 2: 'Хобби и творчество',
 3: 'Товары для дома',
 4: 'Обувь'}

In [61]:
# tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', ngram_range =(1, 2), stop_words=stop_words)

# features = tfidf.fit_transform(corpus_train).toarray()

# labels = lemmatized['level_1_id']

# features.shape

In [62]:
features = tf_idf_train
labels = levels_id['level_1_id']

In [64]:
from sklearn.feature_selection import chi2

N = 4
for Product, category_id in sorted(category_to_id.items()):
    features_chi2 = chi2(features, labels == category_id)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(count_tf_idf.get_feature_names_out())[indices]
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
    #trigrams = [v for v in feature_names if len(v.split(' ')) == 3]
    print("# '{}':".format(Product))
    print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-N:])))
    print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-N:])))
    #print("  . Most correlated trigrams:\n. {}".format('\n. '.join(trigrams[-N:])))

# 'Обувь':
  . Most correlated unigrams:
. стелька
. ботинок
. тапочек
. кроссовок
  . Most correlated bigrams:
. ботинок женский
. кроссовок мужской
. тапочек комнатный
. кроссовок женский
# 'Одежда':
  . Most correlated unigrams:
. трусы
. футболка
. носок
. женский
  . Most correlated bigrams:
. футболка женский
. носок мужской
. носок женский
. трусы женский
# 'Товары для дома':
  . Most correlated unigrams:
. чехол
. женский
. подарочный
. пакет
  . Most correlated bigrams:
. samsung galaxy
. пакет подарочный
. день рождение
. защитный стекло
# 'Хобби и творчество':
  . Most correlated unigrams:
. холст
. подрамник
. картина
. номер
  . Most correlated bigrams:
. алмазный мозаика
. номер холст
. холст подрамник
. картина номер
# 'Электроника':
  . Most correlated unigrams:
. samsung
. защитный
. iphone
. чехол
  . Most correlated bigrams:
. чехол samsung
. xiaomi redmi
. samsung galaxy
. защитный стекло


### Обучение моделей
- Logistic Regression
- (Multinomial) Naive Bayes
- Linear Support Vector Machine
- Random Forest

Таблицы должны быть подготовлены в предыдущей части

Поиск подходящей модели будем осуществлять с момощью RandomSearch

In [65]:
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer

In [66]:
enc = OrdinalEncoder()

In [68]:
ct = make_column_transformer((OrdinalEncoder(), ['level_1', 'level_2', 'level_3', 'level_4']),
                             remainder='passthrough')

In [69]:
a = ct.fit_transform(train)

In [41]:
# Способ декодировать данные трансформера
#a_2 = ct.named_transformers_['ordinalencoder'].inverse_transform(train_coder.loc[:,'title':'level_4'].values)

In [70]:
train_coder = pd.DataFrame(a)

In [71]:
train_coder.head()

,0,1,2,3,4,5,6,7
0,4.0,28.0,18.0,163.0,Зарядный кабель Borofone B...,Кабели,зарядный кабель borofone l...,9031
1,1.0,6.0,35.0,591.0,Трусы Sela,Трусы,трусы sela,18305
2,3.0,25.0,119.0,266.0,"Гуашь ""ЮНЫЙ ВОЛШЕБНИК"", 12...","Краски, пигменты",гуашь юный волшебник цвето...,16357
3,3.0,36.0,97.0,231.0,Колба для кальяна Крафт (р...,Колбы,колба кальян крафт разный ...,34666
4,1.0,6.0,73.0,395.0,"Пижама женская, однотонная...",Пижамы,пижама женский однотонный ...,26389


In [72]:
y_table = train_coder.drop([4,5,6], axis=1).astype('int')

In [73]:
y_table

,0,1,2,3,7
0,4,28,18,163,9031
1,1,6,35,591,18305
2,3,25,119,266,16357
3,3,36,97,231,34666
4,1,6,73,395,26389
...,...,...,...,...,...
91115,4,28,18,166,4955
91116,3,23,21,26,19626
91117,2,38,161,375,22291
91118,4,33,149,463,2985


In [74]:
X = features

In [75]:
LR = LogisticRegression(random_state=12345, multi_class='auto')

In [76]:
%%time
LR.fit(X, y_table[0])

CPU times: total: 812 ms
Wall time: 3.05 s


LogisticRegression(random_state=12345)

In [77]:
y_0 = LR.predict(X)

In [79]:
f1_score(y_table[0], y_0, average='micro')

0.9841856892010535

In [113]:
y_0_0 = y_0.reshape(-1,1)

In [114]:
X_array = X.toarray()

MemoryError: Unable to allocate 18.5 GiB for an array with shape (91120, 27297) and data type float64

In [118]:
X_array[1]

array([0., 0., 0., ..., 0., 0., 0.])

In [88]:
X_2 = np.concatenate([X_array, y_0_0], axis=1)

MemoryError: Unable to allocate 18.5 GiB for an array with shape (91120, 27298) and data type float64

In [119]:
from scipy.sparse import csr_matrix,hstack

In [120]:
S = csr_matrix(y_0_0)

In [94]:
S

<91120x1 sparse matrix of type '<class 'numpy.intc'>'
	with 89536 stored elements in Compressed Sparse Row format>

In [122]:
X_2 = hstack([X, S])

In [124]:
print(type(X_2))
print(X.shape)
print(X_2.shape)

<class 'scipy.sparse._csr.csr_matrix'>
(91120, 27297)
(91120, 27298)


In [125]:
LR_2 = LogisticRegression(random_state=12345, multi_class='auto')

In [127]:
%%time
LR_2.fit(X_2, y_table[1])

CPU times: total: 8.34 s
Wall time: 21.1 s


LogisticRegression(random_state=12345)

In [129]:
y_1 = LR_2.predict(X_2)

In [130]:
f1_score(y_table[1], y_1, average='micro')

0.9115452151009658

In [131]:
y_1_1 = y_1.reshape(-1,1)

In [132]:
S = csr_matrix(y_1_1)

In [134]:
X_3 = hstack([X_2, S])

In [135]:
LR_3 = LogisticRegression(random_state=12345, multi_class='auto')

In [136]:
%%time
LR_3.fit(X_3, y_table[2])

CPU times: total: 45.1 s
Wall time: 1min 59s


LogisticRegression(random_state=12345)

In [137]:
y_2 = LR_3.predict(X_3)

In [138]:
f1_score(y_table[2], y_2, average='micro')

0.39108867427568045

In [139]:
LR_solo = LogisticRegression(random_state=12345, multi_class='auto')

In [140]:
%%time
LR_solo.fit(X, y_table[2])

CPU times: total: 39.7 s
Wall time: 1min 55s


LogisticRegression(random_state=12345)

In [141]:
y_solo = LR_solo.predict(X)

In [142]:
f1_score(y_table[2], y_solo, average='micro')

0.9091966637401229

In [60]:
X_2 = np.concatenate([X, y_1_1], axis=1)

ValueError: zero-dimensional arrays cannot be concatenated

In [63]:
import numpy as np
from scipy.sparse import csr_matrix
from scipy import sparse

In [77]:
A = np.array(X)
print(A.shape, type(A), X.shape)

() <class 'numpy.ndarray'> (91120, 25405)


In [74]:
B = np.array(y_1)
print(B.shape, type(B))

(91120,) <class 'numpy.ndarray'>


In [65]:
np.hstack((X, y_1.reshape(-1,1)))

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 1 has 2 dimension(s)

In [216]:
y_1.shape

(91120,)

In [ ]:
y_1.reshape

In [221]:
type(y_1.reshape(-1,1))

numpy.ndarray

In [75]:
X_array = np.array(X)

In [78]:
X_array.shape

()

In [84]:
pd.DataFrame(a_2).head()

,0,1,2,3,4
0,Электроника,Смартфоны и телефоны,Аксессуары и запчасти,Зарядные устройства и кабели,Кабели
1,Одежда,Женская одежда,Белье и купальники,Трусы,Трусы
2,Хобби и творчество,Рисование,"Краски, пигменты","Краски, пигменты","Краски, пигменты"
3,Хобби и творчество,Товары для курения,Кальяны и аксессуары,Колбы,Колбы
4,Одежда,Женская одежда,Домашняя одежда,Пижамы,Пижамы


Итак, как список последовательных таргетов у нас будут столбцы с уровнями, закодированные с помощью Ordinal Encoder.

In [80]:
train

,title,level_1,level_2,level_3,level_4,level_5,lemm
0,Зарядный кабель Borofone B...,Электроника,Смартфоны и телефоны,Аксессуары и запчасти,Зарядные устройства и кабели,Кабели,зарядный кабель для айфон м\n
1,Трусы Sela,Одежда,Женская одежда,Белье и купальники,Трусы,Трусы,трусы\n
2,"Гуашь ""ЮНЫЙ ВОЛШЕБНИК"", 12...",Хобби и творчество,Рисование,"Краски, пигменты","Краски, пигменты","Краски, пигменты",гуашь юный волшебник цвето...
3,Колба для кальяна Крафт (р...,Хобби и творчество,Товары для курения,Кальяны и аксессуары,Колбы,Колбы,колба для кальян крафт раз...
4,"Пижама женская, однотонная...",Одежда,Женская одежда,Домашняя одежда,Пижамы,Пижамы,пижама женский однотонный ...
...,...,...,...,...,...,...,...
91115,Прочное стекло 2D на Samsu...,Электроника,Смартфоны и телефоны,Аксессуары и запчасти,Защитные стекла и пленки,Защитные стекла,прочный стекло на\n
91116,"Алмазная мозаика ""Ромашки""...",Хобби и творчество,"Пазлы, мозаика и фреска",Алмазные мозаики,Алмазные мозаики,Алмазные мозаики,алмазный мозаика ромашка с...
91117,"Открытка ""Вместе навсегда""...",Товары для дома,Товары для праздников,Открытки и конверты,Открытки,Открытки,открытка вместе навсегда в...
91118,Пульт K10B-C1 для Rolsen,Электроника,Телевизоры и видеотехника,Оборудование для телевизоров,Пульты ДУ,Пульты ДУ,пульт для\n


In [74]:
train_coder

,title,level_1,level_2,level_3,level_4,level_5,lemm
0,4.0,28.0,18.0,163.0,193.0,Зарядный кабель Borofone B...,зарядный кабель для айфон м\n
1,1.0,6.0,35.0,591.0,620.0,Трусы Sela,трусы\n
2,3.0,25.0,119.0,266.0,272.0,"Гуашь ""ЮНЫЙ ВОЛШЕБНИК"", 12...",гуашь юный волшебник цвето...
3,3.0,36.0,97.0,231.0,235.0,Колба для кальяна Крафт (р...,колба для кальян крафт раз...
4,1.0,6.0,73.0,395.0,407.0,"Пижама женская, однотонная...",пижама женский однотонный ...
...,...,...,...,...,...,...,...
91115,4.0,28.0,18.0,166.0,172.0,Прочное стекло 2D на Samsu...,прочный стекло на\n
91116,3.0,23.0,21.0,26.0,29.0,"Алмазная мозаика ""Ромашки""...",алмазный мозаика ромашка с...
91117,2.0,38.0,161.0,375.0,387.0,"Открытка ""Вместе навсегда""...",открытка вместе навсегда в...
91118,4.0,33.0,149.0,463.0,479.0,Пульт K10B-C1 для Rolsen,пульт для\n


,title,level_1,level_2,level_3,level_4,level_5,lemm
0,Зарядный кабель Borofone B...,Электроника,Смартфоны и телефоны,Аксессуары и запчасти,Зарядные устройства и кабели,Кабели,зарядный кабель для айфон м\n
1,Трусы Sela,Одежда,Женская одежда,Белье и купальники,Трусы,Трусы,трусы\n
2,"Гуашь ""ЮНЫЙ ВОЛШЕБНИК"", 12...",Хобби и творчество,Рисование,"Краски, пигменты","Краски, пигменты","Краски, пигменты",гуашь юный волшебник цвето...
3,Колба для кальяна Крафт (р...,Хобби и творчество,Товары для курения,Кальяны и аксессуары,Колбы,Колбы,колба для кальян крафт раз...
4,"Пижама женская, однотонная...",Одежда,Женская одежда,Домашняя одежда,Пижамы,Пижамы,пижама женский однотонный ...
...,...,...,...,...,...,...,...
91115,Прочное стекло 2D на Samsu...,Электроника,Смартфоны и телефоны,Аксессуары и запчасти,Защитные стекла и пленки,Защитные стекла,прочный стекло на\n
91116,"Алмазная мозаика ""Ромашки""...",Хобби и творчество,"Пазлы, мозаика и фреска",Алмазные мозаики,Алмазные мозаики,Алмазные мозаики,алмазный мозаика ромашка с...
91117,"Открытка ""Вместе навсегда""...",Товары для дома,Товары для праздников,Открытки и конверты,Открытки,Открытки,открытка вместе навсегда в...
91118,Пульт K10B-C1 для Rolsen,Электроника,Телевизоры и видеотехника,Оборудование для телевизоров,Пульты ДУ,Пульты ДУ,пульт для\n


In [258]:
X_train = columns_pipeline.fit_transform(lemmatized)

In [259]:
X_train = pd.DataFrame(X_train, columns=[lemmatized.columns])
X_train

,level_1,level_2,level_3,level_4,level_5,lemm,category_id,level_1_id
0,4.0,28.0,18.0,163.0,193.0,зарядный кабель для айфон м\n,12171,0
1,1.0,6.0,35.0,591.0,620.0,трусы\n,14233,1
2,3.0,25.0,119.0,266.0,272.0,гуашь юный волшебник цвето...,13429,2
3,3.0,36.0,97.0,231.0,235.0,колба для кальян крафт раз...,2789,2
4,1.0,6.0,73.0,395.0,407.0,пижама женский однотонный ...,12834,1
...,...,...,...,...,...,...,...,...
91115,4.0,28.0,18.0,166.0,172.0,прочный стекло на\n,14922,0
91116,3.0,23.0,21.0,26.0,29.0,алмазный мозаика ромашка с...,13028,2
91117,2.0,38.0,161.0,375.0,387.0,открытка вместе навсегда в...,13407,3
91118,4.0,33.0,149.0,463.0,479.0,пульт для\n,12100,0


In [265]:
X = features_2
y = X_train['level_1']
y = y.astype('int')

In [200]:
pipeline = make_pipeline(columns_pipeline)
pipeline.steps.append(('clf', DummyRegressor()))
pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat_encoder',
                                                  Pipeline(steps=[('encoder',
                                                                   OrdinalEncoder())]),
                                                  ['level_1', 'level_2',
                                                   'level_3', 'level_4',
                                                   'level_5'])])),
                ('clf', DummyRegressor())])

In [268]:
LR = LogisticRegression(random_state=12345, multi_class='auto')

In [269]:
predict = LR.fit(X, y)

In [274]:
pred = predict.predict(X)

In [278]:
X_train['pred'] = pred

In [279]:
X_train

,level_1,level_2,level_3,level_4,level_5,lemm,category_id,level_1_id,pred
0,4.0,28.0,18.0,163.0,193.0,зарядный кабель для айфон м\n,12171,0,4
1,1.0,6.0,35.0,591.0,620.0,трусы\n,14233,1,1
2,3.0,25.0,119.0,266.0,272.0,гуашь юный волшебник цвето...,13429,2,3
3,3.0,36.0,97.0,231.0,235.0,колба для кальян крафт раз...,2789,2,3
4,1.0,6.0,73.0,395.0,407.0,пижама женский однотонный ...,12834,1,1
...,...,...,...,...,...,...,...,...,...
91115,4.0,28.0,18.0,166.0,172.0,прочный стекло на\n,14922,0,4
91116,3.0,23.0,21.0,26.0,29.0,алмазный мозаика ромашка с...,13028,2,3
91117,2.0,38.0,161.0,375.0,387.0,открытка вместе навсегда в...,13407,3,2
91118,4.0,33.0,149.0,463.0,479.0,пульт для\n,12100,0,4


In [270]:
from sklearn.metrics import f1_score
f1_score(y, pred, average='micro')

In [272]:
predict

LogisticRegression(random_state=12345)

In [277]:
f1_score(y, pred, average='micro')

0.9830223880597015

In [280]:
X_train = columns_pipeline.inverse_transform(y)

AttributeError: 'ColumnTransformer' object has no attribute 'inverse_transform'

In [207]:
LR.get_params().keys()

dict_keys(['C', 'class_weight', 'dual', 'fit_intercept', 'intercept_scaling', 'l1_ratio', 'max_iter', 'multi_class', 'n_jobs', 'penalty', 'random_state', 'solver', 'tol', 'verbose', 'warm_start'])

In [214]:
params = [{
    'multi_class': ['auto', 'ovr', 'multinomial'],
    'max_iter': [i for i in np.arange(50, 200, 15)],
    'solver': ['newton-cg', 'sag', 'saga', 'lbfgs']
    }]

# params = [{
#     'clf': [LogisticRegression(random_state=12345)],
#     'clf__multi_class': ['auto', 'ovr', 'multinomial'],
#     'clf__max_iter': [i for i in np.arange(50, 200, 15)],
#     'clf__solver': ['newton-cg', 'sag', 'saga', 'lbfgs']
#     }]
#     {
#     'clf': [GradientBoostingRegressor(random_state=270323)],
#     'clf__n_estimators': [i for i in np.arange(100, 500, 10)],
#     'clf__max_depth': [i for i in np.arange(1, 15, 2)],
#     'clf__min_samples_split': [i for i in np.arange(4, 14, 2)],
#     'clf__max_features': ['auto', 'sqrt', 'log2'],
#     'columntransformer__num_cols__scaler': [None],
#     'columntransformer__cat_encoder__encoder': [None]
#     },
#     {
#     'clf': [RandomForestRegressor(random_state=270323)],
#     'clf__n_estimators': [i for i in np.arange(150, 500, 15)],
#     'clf__min_samples_split': [i for i in np.arange(4, 14, 2)],
#     'clf__bootstrap': [True, False],
#     'columntransformer__num_cols__scaler': [None],
#     'columntransformer__cat_encoder__encoder': [None]
#     },
#     {
#     'clf': [CatBoostRegressor(random_state=270323, verbose=0)],
#     'clf__n_estimators': estimators_range,
#     'clf__max_depth': max_depth_range,
#     'columntransformer__num_cols__scaler': [None],
#     'columntransformer__cat_encoder__encoder': [None]
#     }]

In [215]:
grid = RandomizedSearchCV(LR,
                    params,
                    n_iter=100,
                    cv=3,
                    verbose=2,
                    random_state=12345,
                    scoring='f1',
                    n_jobs=-1)

In [216]:
grid.fit(features_2, y)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] END .....max_iter=50, multi_class=ovr, solver=newton-cg; total time=   0.0s
[CV] END .....max_iter=50, multi_class=ovr, solver=newton-cg; total time=   0.0s
[CV] END .....max_iter=50, multi_class=ovr, solver=newton-cg; total time=   0.0s
[CV] END .........max_iter=65, multi_class=ovr, solver=lbfgs; total time=   0.0s
[CV] END .........max_iter=65, multi_class=ovr, solver=lbfgs; total time=   0.0s
[CV] END .........max_iter=65, multi_class=ovr, solver=lbfgs; total time=   0.0s
[CV] END ...max_iter=80, multi_class=multinomial, solver=sag; total time=   0.0s
[CV] END ...max_iter=80, multi_class=multinomial, solver=sag; total time=   0.0s
[CV] END ...max_iter=80, multi_class=multinomial, solver=sag; total time=   0.0s
[CV] END ...max_iter=155, multi_class=auto, solver=newton-cg; total time=   0.0s
[CV] END ...max_iter=155, multi_class=auto, solver=newton-cg; total time=   0.0s
[CV] END ...max_iter=155, multi_class=auto, so

[CV] END ...max_iter=125, multi_class=auto, solver=newton-cg; total time=   0.0s
[CV] END ...max_iter=125, multi_class=auto, solver=newton-cg; total time=   0.0s
[CV] END ........max_iter=185, multi_class=auto, solver=saga; total time=   0.0s
[CV] END ........max_iter=185, multi_class=auto, solver=saga; total time=   0.0s
[CV] END ........max_iter=185, multi_class=auto, solver=saga; total time=   0.0s
[CV] END .........max_iter=155, multi_class=ovr, solver=saga; total time=   0.0s
[CV] END .........max_iter=155, multi_class=ovr, solver=saga; total time=   0.0s
[CV] END .........max_iter=155, multi_class=ovr, solver=saga; total time=   0.0s
[CV] END max_iter=125, multi_class=multinomial, solver=newton-cg; total time=   0.0s
[CV] END max_iter=125, multi_class=multinomial, solver=newton-cg; total time=   0.0s
[CV] END max_iter=125, multi_class=multinomial, solver=newton-cg; total time=   0.0s
[CV] END max_iter=50, multi_class=multinomial, solver=newton-cg; total time=   0.0s
[CV] END max_

[CV] END max_iter=170, multi_class=multinomial, solver=lbfgs; total time=   0.0s
[CV] END max_iter=170, multi_class=multinomial, solver=lbfgs; total time=   0.0s
[CV] END ...max_iter=140, multi_class=auto, solver=newton-cg; total time=   0.0s
[CV] END ...max_iter=140, multi_class=auto, solver=newton-cg; total time=   0.0s
[CV] END ...max_iter=140, multi_class=auto, solver=newton-cg; total time=   0.0s
[CV] END ..........max_iter=110, multi_class=ovr, solver=sag; total time=   0.0s
[CV] END ..........max_iter=110, multi_class=ovr, solver=sag; total time=   0.0s
[CV] END ..........max_iter=110, multi_class=ovr, solver=sag; total time=   0.0s
[CV] END ....max_iter=185, multi_class=ovr, solver=newton-cg; total time=   0.0s
[CV] END ....max_iter=185, multi_class=ovr, solver=newton-cg; total time=   0.0s
[CV] END ....max_iter=185, multi_class=ovr, solver=newton-cg; total time=   0.0s
[CV] END .........max_iter=155, multi_class=auto, solver=sag; total time=   0.0s
[CV] END .........max_iter=1

ValueError: Unknown label type: 'unknown'

Источник для дальнейшей категоризации

https://towardsdatascience.com/multi-class-text-classification-with-scikit-learn-12f1e60e0a9f

Модели полюс нейронка, узнать про написание своего ансамбля
https://www.kaggle.com/code/abhishek/approaching-almost-any-nlp-problem-on-kaggle/notebook

Что дальше?

Всё же продолжить работу с тысячей строк. Опробовать модели через пайплайн